In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [3]:
admissions = pd.read_csv('demo_data/ADMISSIONS.csv.gz', compression = 'gzip')
noteevents = pd.read_csv('demo_data/NOTEEVENTS.csv.gz', compression = 'gzip')

/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_55261/1709349325.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteevents = pd.read_csv('demo_data/NOTEEVENTS.csv.gz', compression = 'gzip')


In [4]:
admissions.ADMITTIME = pd.to_datetime(admissions.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
admissions.DISCHTIME = pd.to_datetime(admissions.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
admissions.DEATHTIME = pd.to_datetime(admissions.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

admissions = admissions.sort_values(['SUBJECT_ID','ADMITTIME'])
admissions = admissions.reset_index(drop = True)
admissions['NEXT_ADMITTIME'] = admissions.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
admissions['NEXT_ADMISSION_TYPE'] = admissions.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

rows = admissions.NEXT_ADMISSION_TYPE == 'ELECTIVE'
admissions.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
admissions.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

admissions = admissions.sort_values(['SUBJECT_ID','ADMITTIME'])

#When we filter out the "ELECTIVE", we need to correct the next admit time for these admissions since there might be 'emergency' next admit after "ELECTIVE"
admissions[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = admissions.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
admissions['DAYS_NEXT_ADMIT']=  (admissions.NEXT_ADMITTIME - admissions.DISCHTIME).dt.total_seconds()/(24*60*60)
admissions['OUTPUT_LABEL'] = (admissions.DAYS_NEXT_ADMIT < 30).astype('int')
### filter out newborn and death
admissions = admissions[admissions['ADMISSION_TYPE']!='NEWBORN']
admissions = admissions[admissions.DEATHTIME.isnull()]
admissions['DURATION'] = (admissions['DISCHTIME']-admissions['ADMITTIME']).dt.total_seconds()/(24*60*60)

/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_55261/1421443384.py:17: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use DataFrame.fillna instead
  admissions[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = admissions.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_55261/1421443384.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  admissions[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = admissions.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_55261/1421443384.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .f

In [5]:
noteevents = noteevents.sort_values(by=['SUBJECT_ID','HADM_ID','CHARTDATE'])
adm_notes = pd.merge(admissions[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME','OUTPUT_LABEL','DURATION']],
                     noteevents[['SUBJECT_ID','HADM_ID','CHARTDATE','TEXT','CATEGORY']], 
                     on = ['SUBJECT_ID','HADM_ID'],
                     how = 'left')

In [6]:
adm_notes_copy = adm_notes.copy()
adm_notes = adm_notes.sample(1000)

In [7]:
adm_notes.ADMITTIME_C = adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])
adm_notes['ADMITTIME_C'] = pd.to_datetime(adm_notes.ADMITTIME_C, format = '%Y-%m-%d', errors = 'coerce')
adm_notes['CHARTDATE'] = pd.to_datetime(adm_notes.CHARTDATE, format = '%Y-%m-%d', errors = 'coerce')

### If Discharge Summary 
discharge = adm_notes[adm_notes['CATEGORY'] == 'Discharge summary']
# multiple discharge summary for one admission -> after examination -> replicated summary -> replace with the last one 
discharge = (discharge.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()
discharge = discharge[discharge['TEXT'].notnull()]

### If Less than n days on admission notes (Early notes)
def less_n_days_data (adm_notes, n):
    
    less_n = adm_notes[((adm_notes['CHARTDATE']-adm_notes['ADMITTIME_C']).dt.total_seconds()/(24*60*60))<n]
    less_n=less_n[less_n['TEXT'].notnull()]
    #concatenate first
    concat = pd.DataFrame(less_n.groupby('HADM_ID')['TEXT'].apply(lambda x: "%s" % ' '.join(x))).reset_index()
    concat['OUTPUT_LABEL'] = concat['HADM_ID'].apply(lambda x: less_n[less_n['HADM_ID']==x].OUTPUT_LABEL.values[0])

    return concat

less_2 = less_n_days_data(adm_notes, 2)
less_3 = less_n_days_data(adm_notes, 3)

/var/folders/92/pwtffhb94b3526gcw6x7zlqw0000gn/T/ipykernel_55261/2455160033.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  adm_notes.ADMITTIME_C = adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])


In [8]:
def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    return y

def preprocessing(df_less_n): 
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: preprocess1(x))

    df_len = len(df_less_n)
    want=pd.DataFrame({'ID':[],'TEXT':[],'Label':[]})
    for i in tqdm(range(df_len)):
        x=df_less_n.TEXT.iloc[i].split()
        n=int(len(x)/318)
        for j in range(n):
            new_row = pd.DataFrame({'TEXT':' '.join(x[j*318:(j+1)*318]),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]}, index=[0])
            want=pd.concat([want, new_row],ignore_index=True)
        if len(x)%318>10:
            new_row = pd.DataFrame({'TEXT':' '.join(x[-(len(x)%318):]),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]}, index=[0])
            want=pd.concat([want, new_row],ignore_index=True)
    
    return want

In [9]:
discharge = preprocessing(discharge)
less_2 = preprocessing(less_2)
less_3 = preprocessing(less_3)

100%|██████████| 392/392 [00:00<00:00, 1573.59it/s]


In [15]:
discharge['TEXT'][0]

'date of birth: sex: f service: medicine allergies: methadone attending: chief complaint: neck pain, feeling "awful" major surgical or invasive procedure: placement of triple lumen catheter in right ij history of present illness: 56 year old woman with dm, htn, esrd on hd, recent mva, who presents with neck pain and feeling "awful." she was involved in a mva two weeks ago when another car struck her car from the passenger side. she was in the passenger seat, wearing a seatbelt, did not strike her head but did experience a jerking motion of her neck. afterward she began to experience severe headaches and posterior neck pain. she was seen at on and diagnosed with whiplash. x-rays were ordered which she did not have done. she went to hospital yesterday for the pain and was given dilaudid and ativan and discharged home. she continued to feel unwell so came to the ed. in the ed she initially triggered for hypoxia of 82% on ra in triage, but on recheck was 100% on ra. other vs were 0, 47, 94

In [11]:
readmit_ID = admissions[admissions.OUTPUT_LABEL == 1].HADM_ID
not_readmit_ID = admissions[admissions.OUTPUT_LABEL == 0].HADM_ID
#subsampling to get the balanced pos/neg numbers of patients for each dataset
not_readmit_ID_use = not_readmit_ID.sample(n=len(readmit_ID), random_state=1)
id_val_test_t=readmit_ID.sample(frac=0.2,random_state=1)
id_val_test_f=not_readmit_ID_use.sample(frac=0.2,random_state=1)

id_train_t = readmit_ID.drop(id_val_test_t.index)
id_train_f = not_readmit_ID_use.drop(id_val_test_f.index)

id_val_t=id_val_test_t.sample(frac=0.5,random_state=1)
id_test_t=id_val_test_t.drop(id_val_t.index)

id_val_f=id_val_test_f.sample(frac=0.5,random_state=1)
id_test_f=id_val_test_f.drop(id_val_f.index)

# test if there is overlap between train and test, should return "array([], dtype=int64)"
(pd.Index(id_test_t).intersection(pd.Index(id_train_t))).values

id_test = pd.concat([id_test_t, id_test_f])
test_id_label = pd.DataFrame(data = list(zip(id_test, [1]*len(id_test_t)+[0]*len(id_test_f))), columns = ['id','label'])

id_val = pd.concat([id_val_t, id_val_f])
val_id_label = pd.DataFrame(data = list(zip(id_val, [1]*len(id_val_t)+[0]*len(id_val_f))), columns = ['id','label'])

id_train = pd.concat([id_train_t, id_train_f])
train_id_label = pd.DataFrame(data = list(zip(id_train, [1]*len(id_train_t)+[0]*len(id_train_f))), columns = ['id','label'])

#get discharge train/val/test

discharge_train = discharge[discharge.ID.isin(train_id_label.id)]
discharge_val = discharge[discharge.ID.isin(val_id_label.id)]
discharge_test = discharge[discharge.ID.isin(test_id_label.id)]

# subsampling for training....since we obtain training on patient admission level so now we have same number of pos/neg readmission
# but each admission is associated with different length of notes and we train on each chunks of notes, not on the admission, we need
# to balance the pos/neg chunks on training set. (val and test set are fine) Usually, positive admissions have longer notes, so we need 
# find some negative chunks of notes from not_readmit_ID that we haven't used yet

df = pd.concat([not_readmit_ID_use, not_readmit_ID])
df = df.drop_duplicates(keep=False)
#check to see if there are overlaps
(pd.Index(df).intersection(pd.Index(not_readmit_ID_use))).values

# for this set of split with random_state=1, we find we need 400 more negative training samples
not_readmit_ID_more = df.sample(n=400, random_state=1)
discharge_train_snippets = pd.concat([discharge[discharge.ID.isin(not_readmit_ID_more)], discharge_train])

#shuffle
discharge_train_snippets = discharge_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

#check if balanced
discharge_train_snippets.Label.value_counts()

discharge_train_snippets.to_csv('./discharge_train.csv')
discharge_val.to_csv('./discharge_val.csv')
discharge_test.to_csv('./discharge_test.csv')

### for Early notes experiment: we only need to find training set for 3 days, then we can test 
### both 3 days and 2 days. Since we split the data on patient level and experiments share admissions
### in order to see the progression, the 2 days training dataset is a subset of 3 days training set.
### So we only train 3 days and we can test/val on both 2 & 3days or any time smaller than 3 days. This means
### if we train on a dataset with all the notes in n days, we can predict readmissions smaller than n days. 

#for 3 days note, similar to discharge

early_train = less_3[less_3.ID.isin(train_id_label.id)]
not_readmit_ID_more = df.sample(n=500, random_state=1)
early_train_snippets = pd.concat([less_3[less_3.ID.isin(not_readmit_ID_more)], early_train])
#shuffle
early_train_snippets = early_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)
early_train_snippets.to_csv('./3days_train.csv')

early_val = less_3[less_3.ID.isin(val_id_label.id)]
early_val.to_csv('./3days_val.csv')

# we want to test on admissions that are not discharged already. So for less than 3 days of notes experiment,
# we filter out admissions discharged within 3 days
actionable_ID_3days = admissions[admissions['DURATION'] >= 3].HADM_ID
test_actionable_id_label = test_id_label[test_id_label.id.isin(actionable_ID_3days)]
early_test = less_3[less_3.ID.isin(test_actionable_id_label.id)]

early_test.to_csv('./3days_test.csv')

#for 2 days notes, we only obtain test set. Since the model parameters are tuned on the val set of 3 days

actionable_ID_2days = admissions[admissions['DURATION'] >= 2].HADM_ID

test_actionable_id_label_2days = test_id_label[test_id_label.id.isin(actionable_ID_2days)]

early_test_2days = less_2[less_2.ID.isin(test_actionable_id_label_2days.id)]

early_test_2days.to_csv('./2days_test.csv')